<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/ShaliniAnandaPhD/blob/main/Summarization%2B%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tika

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=dd5eb927bb286f4dcaff5a937698b5a245e000c46d01fee96260efbbf87a4df0
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [5]:
import urllib.request

def download_pdf(url, file_path):
  response = urllib.request.urlopen(url)
  pdf_content = response.read()

  with open(file_path, "wb") as f:
    f.write(pdf_content)

In [6]:
from tika import parser
import spacy
from scipy.spatial.distance import cosine

nlp = spacy.load('en_core_web_sm')

def parse_pdf_content(file_path):
  parsed_pdf = parser.from_file(file_path)
  return parsed_pdf['content'].strip()

def chunk_text(text, chunk_size=200, overlap=50):
  tokens = text.split()
  for i in range(0, len(tokens), chunk_size-overlap):
    yield tokens[i:i+chunk_size]

def get_text_vectors(text, chunk_size, overlap):
  chunks = chunk_text(text, chunk_size, overlap)
  return [nlp(" ".join(chunk)) for chunk in chunks]

def combine_semantic_chunks(vector_chunks, text_chunks):
  paragraphs = [text_chunks[0]]

  for i in range(1, len(vector_chunks)):
    similarity = cosine(vector_chunks[i-1], vector_chunks[i])
    if similarity < 0.8:
      paragraphs.append(text_chunks[i])
    else:
      paragraphs[-1].extend(text_chunks[i])

  return paragraphs

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

def score_sentences(sentences):
  freqs = {}
  for sent in sentences:
    for word in word_tokenize(sent):
      if word not in stopwords.words('english'):
        freqs[word] = freqs.get(word, 0) + 1

  max_freq = max(freqs.values())
  for word in freqs:
    freqs[word] /= max_freq

  sent_scores = {sent:0 for sent in sentences}
  for sent in sentences:
    for word in word_tokenize(sent):
      if word in freqs:
        sent_scores[sent] += freqs[word]

  return sent_scores

def summarize(text, threshold=0.8):
  sentences = sent_tokenize(text)
  sent_scores = score_sentences(sentences)

  summary = []
  for sent, score in sent_scores.items():
    if score > threshold:
      summary.append(sent)

  return " ".join(summary)

In [18]:
# Imports
import requests
from tika import parser
import spacy
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Global Variables
nlp = spacy.load('en_core_web_sm')

# PDF Downloader
def download_pdf(url, file_path):
    response = requests.get(url)
    content = response.content

    with open(file_path, 'wb') as f:
        f.write(content)

# PDF Parsing
def parse_pdf_content(file_path):
    parsed = parser.from_file(file_path)
    return parsed['content']

def chunk_pdf_text(text, chunk_size, overlap):
    tokens = text.split()

    for i in range(0, len(tokens), chunk_size - overlap):
        yield " ".join(tokens[i:i + chunk_size])

def vectorize_chunks(chunks):
    return [nlp(chunk) for chunk in chunks]

# Semantic Grouping
def group_semantic_chunks(vector_chunks, text_chunks):
    paragraphs = [text_chunks[0]]

    for i in range(1, len(vector_chunks)):
        similarity = 1 - cosine(vector_chunks[i-1].vector, vector_chunks[i].vector)

        if similarity < 0.8:
            paragraphs.append(text_chunks[i])

        else:
            paragraphs[-1] += " " + text_chunks[i]

    return paragraphs

# Summarization
def score_sentences(sentences):
    # Score logic
    # TODO: Implement the score logic
    pass

def summarize(text, threshold):
    # Summarize logic
    # TODO: Implement the summarize logic
    return text

# Main Function
def summarize_pdf(url, file):
    # Download
    download_pdf(url, file)

    # Parse
    text = parse_pdf_content(file)

    # Detect Semantic Chunks
    chunks = list(chunk_pdf_text(text, 200, 50))
    vectors = vectorize_chunks(chunks)

    # Group
    paragraphs = group_semantic_chunks(vectors, chunks)

    # Summarize
    summaries = []

    for para in paragraphs:
        summary = summarize(para, 0.8)
        summaries.append(summary)

    return summaries

# Driver
url = 'https://www.uky.edu/Projects/Chemcomics/'
summaries = summarize_pdf(url, 'sample.pdf')
print(summaries)


2023-09-12 14:12:02,191 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-09-12 14:12:02,696 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-09-12 14:12:03,143 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-09-12 14:12:08,153 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


["The Comic Book Periodic Table of the Elements Welcome to the Periodic Table of Comic Books. Click on an element to see a list of comic book pages involvingthat element. Click on a thumbnail on the list to see a full comic bookpage. For technical information about an element, follow the link to Mark Winter's WebElements. We recommend that you start with oxygen to see some of our best stuff. There's something for everyone here! Over 6,000,000 Hits! John P. Selegue F. James Holler Department of Chemistry, Universityof Kentucky Lexington,KY 40506-0055 Version 2.012 This document and associatedimages ©1996-2012, F. James Holler and John P.Selegue. All rights reserved.."]
